In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[3]: [FileInfo(path='dbfs:/FileStore/tables/employees.csv', name='employees.csv', size=3778, modificationTime=1688316609000),
 FileInfo(path='dbfs:/FileStore/tables/hello.txt', name='hello.txt', size=22, modificationTime=1675083047000),
 FileInfo(path='dbfs:/FileStore/tables/test/', name='test/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/test.csv', name='test.csv', size=71, modificationTime=1675082472000)]

In [0]:
df = spark.read.format('csv')\
.option('mode','FAILFAST')\
.option('header','true')\
.option('inferSchema','True')\
.load('/FileStore/tables/employees.csv')

In [0]:
#df.printSchema()
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
df.columns

Out[14]: ['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [0]:
from pyspark.sql.types import StructType , IntegerType ,StringType, DateType, FloatType, StructField
from pyspark.sql.functions import col

In [0]:
#build schema 

my_schema = StructType(
[
    StructField("EMPLOYEE_ID",IntegerType(),True),
    StructField("FIRST_NAME",StringType(),True),
    StructField("LAST_NAME",StringType(),True),
    StructField("EMAIL",StringType(),True),
    StructField("PHONE_NUMBER",IntegerType(),True),
    StructField("HIRE_DATE",DateType(),True),
    StructField("JOB_ID",StringType(),True),
    StructField("SALARY",IntegerType(),True),
    StructField("COMMISSION_PCT",FloatType(),True),
    StructField("MANAGER_ID",IntegerType(),True),
    StructField("DEPARTMENT_ID",IntegerType(),True)

]
)

In [0]:
# DataFrame - consist of Rows and Columns. Columns are expression ( means set of transformation on one or more than one values in a row)
# Select columns by string method 

df.select("FIRST_NAME").show(2)

+----------+
|FIRST_NAME|
+----------+
|    Donald|
|   Douglas|
+----------+
only showing top 2 rows



In [0]:
# Select by col method. 
df.select(col("FIRST_NAME")).show(2)

+----------+
|FIRST_NAME|
+----------+
|    Donald|
|   Douglas|
+----------+
only showing top 2 rows



In [0]:
# manipulating column by col method 
df.select(col("EMPLOYEE_ID")+5).show(2)

+-----------------+
|(EMPLOYEE_ID + 5)|
+-----------------+
|              203|
|              204|
+-----------------+
only showing top 2 rows



Multiple column select

In [0]:
df.select("EMPLOYEE_ID","FIRST_NAME","LAST_NAME").show(2)

+-----------+----------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|
+-----------+----------+---------+
|        198|    Donald| OConnell|
|        199|   Douglas|    Grant|
+-----------+----------+---------+
only showing top 2 rows



In [0]:
df.select(col("EMPLOYEE_ID"),col("FIRST_NAME"),col("LAST_NAME")).show(2)

+-----------+----------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|
+-----------+----------+---------+
|        198|    Donald| OConnell|
|        199|   Douglas|    Grant|
+-----------+----------+---------+
only showing top 2 rows



In [0]:
#Different way to select columns. the bracket way are used when we are joining dataframe when column name are different.
df.select("EMPLOYEE_ID",col("FIRST_NAME"),df["LAST_NAME"],df.EMAIL).show(2)

+-----------+----------+---------+--------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|
+-----------+----------+---------+--------+
|        198|    Donald| OConnell|DOCONNEL|
|        199|   Douglas|    Grant|  DGRANT|
+-----------+----------+---------+--------+
only showing top 2 rows



Expression

In [0]:
df.select("EMPLOYEE_ID + 5").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1692221140788134>:1
----> 1 df.select("EMPLOYEE_ID +5").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a new :class:`DataFrame`.
   2980 
   2981     .. versionadded:: 1.3.0
   (...)
   3021     +-----+---+
   3022     """
-> 3023     jdf = self._jdf.select(self.

In [0]:
df.select(expr("EMPLOYEE_ID + 5 ")).show(2)

+-----------------+
|(EMPLOYEE_ID + 5)|
+-----------------+
|              203|
|              204|
+-----------------+
only showing top 2 rows



In [0]:
df.select(expr("EMPLOYEE_ID as emp_id"),expr("FIRST_NAME as first_name"), expr("concat(FIRST_NAME,LAST_NAME) as full_name")).show(5)

+------+----------+----------------+
|emp_id|first_name|       full_name|
+------+----------+----------------+
|   198|    Donald|  DonaldOConnell|
|   199|   Douglas|    DouglasGrant|
|   200|  Jennifer|  JenniferWhalen|
|   201|   Michael|MichaelHartstein|
|   202|       Pat|          PatFay|
+------+----------+----------------+
only showing top 5 rows



In [0]:
df.select("*").show(2)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 2 rows



Spark SQL 

In [0]:
# To convert the dataframe into sql we have to table or temp view 

df.createOrReplaceTempView("emp")

In [0]:
spark.sql(""" 
select * from emp limit 10          
""").show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


Transformation

In [0]:
# Filter/ where 
df.display()

EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50
199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50
200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,-,101,10
201,Michael,Hartstein,MHARTSTE,515.123.5555,17-FEB-04,MK_MAN,13000,-,100,20
202,Pat,Fay,PFAY,603.123.6666,17-AUG-05,MK_REP,6000,-,201,20
203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,-,101,40
204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,-,101,70
205,Shelley,Higgins,SHIGGINS,515.123.8080,07-JUN-02,AC_MGR,12008,-,101,110
206,William,Gietz,WGIETZ,515.123.8181,07-JUN-02,AC_ACCOUNT,8300,-,205,110
100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,-,-,90


In [0]:
df.filter((col("salary") > 3000) & (col("employee_id")> 205)).display()

EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
206,William,Gietz,WGIETZ,515.123.8181,07-JUN-02,AC_ACCOUNT,8300,-,205,110


In [0]:
# aliasing

df.select(col("employee_id").alias("emp_id")).display()

emp_id
198
199
200
201
202
203
204
205
206
100


In [0]:
from pyspark.sql.functions import *

In [0]:
# column add, col rename, 
df.withColumn("FULL_NAME",concat(col("first_name"),col("last_name"))).display()

EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID,FULL_NAME
198,Donald,OConnell,DOCONNEL,650.507.9833,21-JUN-07,SH_CLERK,2600,-,124,50,DonaldOConnell
199,Douglas,Grant,DGRANT,650.507.9844,13-JAN-08,SH_CLERK,2600,-,124,50,DouglasGrant
200,Jennifer,Whalen,JWHALEN,515.123.4444,17-SEP-03,AD_ASST,4400,-,101,10,JenniferWhalen
201,Michael,Hartstein,MHARTSTE,515.123.5555,17-FEB-04,MK_MAN,13000,-,100,20,MichaelHartstein
202,Pat,Fay,PFAY,603.123.6666,17-AUG-05,MK_REP,6000,-,201,20,PatFay
203,Susan,Mavris,SMAVRIS,515.123.7777,07-JUN-02,HR_REP,6500,-,101,40,SusanMavris
204,Hermann,Baer,HBAER,515.123.8888,07-JUN-02,PR_REP,10000,-,101,70,HermannBaer
205,Shelley,Higgins,SHIGGINS,515.123.8080,07-JUN-02,AC_MGR,12008,-,101,110,ShelleyHiggins
206,William,Gietz,WGIETZ,515.123.8181,07-JUN-02,AC_ACCOUNT,8300,-,205,110,WilliamGietz
100,Steven,King,SKING,515.123.4567,17-JUN-03,AD_PRES,24000,-,-,90,StevenKing


In [0]:
df.withColumn("FIRST_NAME",upper(col("first_name"))).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    DONALD| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   DOUGLAS|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  JENNIFER|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   MICHAEL|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       PAT|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
# Rename Column

df.withColumnRenamed("employee_id","EMP_ID").show()

+------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMP_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|   198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|   199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|   200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|   201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|   202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|
|   203|     Susan|   Mavris| SMAVRIS|51

In [0]:
# Catsing a column
df.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [0]:
# date type formating
df_change = df.withColumn("HIRE_DATE",to_date(col("HIRE_DATE")))

In [0]:
df_change.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: date (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [0]:
# Casting dataframe type 
df.withColumn("MANAGER_ID",col("MANAGER_ID").cast("integer")).printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [0]:
# Remove columns 
df.drop(col("manager_id"),"department_id").show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|07-JUN-02|    HR_REP|  6500|            - |
|        204|   Hermann|     Baer|   HBAER|515.123.8888|07-JUN-02|    PR_REP| 10000|            - |


In [0]:
# literal - this will add the value to all the columns 
df.withColumn("extra_col",lit("Unknown")).show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|extra_col|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+---------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|  Unknown|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|  Unknown|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|  Unknown|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|  Unknown|
|        202|       Pat|      Fay|    PFAY|603.123.6666

Aggregate fun and Group by

In [0]:
df_new = df.select("employee_id","first_name","email","salary","manager_id","DEPARTMENT_ID")

In [0]:
df_new.show()

+-----------+----------+--------+------+----------+-------------+
|employee_id|first_name|   email|salary|manager_id|DEPARTMENT_ID|
+-----------+----------+--------+------+----------+-------------+
|        198|    Donald|DOCONNEL|  2600|       124|           50|
|        199|   Douglas|  DGRANT|  2600|       124|           50|
|        200|  Jennifer| JWHALEN|  4400|       101|           10|
|        201|   Michael|MHARTSTE| 13000|       100|           20|
|        202|       Pat|    PFAY|  6000|       201|           20|
|        203|     Susan| SMAVRIS|  6500|       101|           40|
|        204|   Hermann|   HBAER| 10000|       101|           70|
|        205|   Shelley|SHIGGINS| 12008|       101|          110|
|        206|   William|  WGIETZ|  8300|       205|          110|
|        100|    Steven|   SKING| 24000|        - |           90|
|        101|     Neena|NKOCHHAR| 17000|       100|           90|
|        102|       Lex| LDEHAAN| 17000|       100|           90|
|        1

In [0]:
# aggregare fun 
df1 = df_new.select(count("employee_id").alias("Total No of employee"), sum("salary").alias("Total_Salary"), max("salary").alias("Max_Salary"),
              min("salary").alias("Min_salary"))

In [0]:
df1.display()

Total No of employee,Total_Salary,Max_Salary,Min_salary
50,309116,24000,2100


In [0]:
df1.collect()[0][0]

Out[75]: 50

In [0]:
# Group by 

df_new.groupBy("DEPARTMENT_ID").agg(sum(col("salary")).alias("total_sal"),count("salary").alias("no of rec"),max("salary").alias("Max sal"),min("salary").alias("min_sal")).show()

+-------------+---------+---------+-------+-------+
|DEPARTMENT_ID|total_sal|no of rec|Max sal|min_sal|
+-------------+---------+---------+-------+-------+
|           20|    19000|        2|  13000|   6000|
|           40|     6500|        1|   6500|   6500|
|          100|    51608|        6|  12008|   6900|
|           10|     4400|        1|   4400|   4400|
|           50|    85600|       23|   8200|   2100|
|           70|    10000|        1|  10000|  10000|
|           90|    58000|        3|  24000|  17000|
|           60|    28800|        5|   9000|   4200|
|          110|    20308|        2|  12008|   8300|
|           30|    24900|        6|  11000|   2500|
+-------------+---------+---------+-------+-------+



In [0]:
# New data set 
datas = [(1,"manish",50000,"IT","india"),
(2,"vikash",60000,"sales","us"),
(3,"raushan",70000,"marketing","india"),
(4,"mukesh",80000,"IT","us"),
(5,"pritam",90000,"sales","india"),
(6,"nikita",45000,"marketing","us"),
(7,"ragini",55000,"marketing","india"),
(8,"rakesh",100000,"IT","us"),
(9,"aditya",65000,"IT","india"),
(10,"rahul",50000,"marketing","us")]

my_schema = ["Emp_id","emp_name","sal","dept","country"]

df = spark.createDataFrame(data=datas,schema=my_schema) 

In [0]:
df.show()

+------+--------+------+---------+-------+
|Emp_id|emp_name|   sal|     dept|country|
+------+--------+------+---------+-------+
|     1|  manish| 50000|       IT|  india|
|     2|  vikash| 60000|    sales|     us|
|     3| raushan| 70000|marketing|  india|
|     4|  mukesh| 80000|       IT|     us|
|     5|  pritam| 90000|    sales|  india|
|     6|  nikita| 45000|marketing|     us|
|     7|  ragini| 55000|marketing|  india|
|     8|  rakesh|100000|       IT|     us|
|     9|  aditya| 65000|       IT|  india|
|    10|   rahul| 50000|marketing|     us|
+------+--------+------+---------+-------+



In [0]:
df.groupBy(col("dept"),col("country")).agg(sum("sal").alias('tot_sal')).orderBy(col("dept"),col("tot_sal").desc()).show()

+---------+-------+-------+
|     dept|country|tot_sal|
+---------+-------+-------+
|       IT|     us| 180000|
|       IT|  india| 115000|
|marketing|  india| 125000|
|marketing|     us|  95000|
|    sales|  india|  90000|
|    sales|     us|  60000|
+---------+-------+-------+



Joins

In [0]:
# Preparing data sets for joining

empData = [(1,"Smith",10), (2,"Rose",20),
    (3,"Williams",10), (4,"Jones",30)
  ]
empColumns = ["emp_id","name","emp_dept_id"]
empDF = spark.createDataFrame(empData,empColumns)
empDF.show()

deptData = [("Finance",10), ("Marketing",20),
    ("Sales",30),("IT",40)
  ]
deptColumns = ["dept_name","dept_id"]
deptDF=spark.createDataFrame(deptData,deptColumns)  
deptDF.show()

addData=[(1,"1523 Main St","SFO","CA"),
    (2,"3453 Orange St","SFO","NY"),
    (3,"34 Warner St","Jersey","NJ"),
    (4,"221 Cavalier St","Newark","DE"),
    (5,"789 Walnut St","Sandiago","CA")
  ]
addColumns = ["emp_id","addline1","city","state"]
addDF = spark.createDataFrame(addData,addColumns)
addDF.show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     1|   Smith|         10|
|     2|    Rose|         20|
|     3|Williams|         10|
|     4|   Jones|         30|
+------+--------+-----------+

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+

+------+---------------+--------+-----+
|emp_id|       addline1|    city|state|
+------+---------------+--------+-----+
|     1|   1523 Main St|     SFO|   CA|
|     2| 3453 Orange St|     SFO|   NY|
|     3|   34 Warner St|  Jersey|   NJ|
|     4|221 Cavalier St|  Newark|   DE|
|     5|  789 Walnut St|Sandiago|   CA|
+------+---------------+--------+-----+



In [0]:
# Join -> Inner Join
empDF.join(deptDF,empDF["emp_dept_id"] == deptDF["dept_id"],'inner').show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     3|Williams|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     4|   Jones|         30|    Sales|     30|
+------+--------+-----------+---------+-------+



In [0]:
# Join -> Left outer Join
empDF.join(deptDF,empDF["emp_dept_id"] == deptDF["dept_id"],'left').show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     3|Williams|         10|  Finance|     10|
|     4|   Jones|         30|    Sales|     30|
+------+--------+-----------+---------+-------+



In [0]:
# Join -> Right outer Join
empDF.join(deptDF,empDF["emp_dept_id"] == deptDF["dept_id"],'right').show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     3|Williams|         10|  Finance|     10|
|     1|   Smith|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     4|   Jones|         30|    Sales|     30|
|  null|    null|       null|       IT|     40|
+------+--------+-----------+---------+-------+



In [0]:
# Join ->  outer Join
empDF.join(deptDF,empDF["emp_dept_id"] == deptDF["dept_id"],'outer').show()

+------+--------+-----------+---------+-------+
|emp_id|    name|emp_dept_id|dept_name|dept_id|
+------+--------+-----------+---------+-------+
|     1|   Smith|         10|  Finance|     10|
|     3|Williams|         10|  Finance|     10|
|     2|    Rose|         20|Marketing|     20|
|     4|   Jones|         30|    Sales|     30|
|  null|    null|       null|       IT|     40|
+------+--------+-----------+---------+-------+



Windows Functions - > rank, dense rank and row number

In [0]:
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import *

In [0]:
win =  Window.partitionBy("job_id",).orderBy("salary")
df.withColumn("rank",row_number().over(win)).orderBy("job_id").show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|rank|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----+
|        206|    William|    Gietz|  WGIETZ|515.123.8181|07-JUN-02|AC_ACCOUNT|  8300|            - |       205|          110|   1|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|          110|   1|
|        200|   Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|   1|
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|   1|
|        101|      Neena|  Kochhar|NKOCHHAR|515.123.4568|21-SEP-05|     AD_VP| 1700

In [0]:
df.withColumn("dense_rank",dense_rank().over(win)).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|dense_rank|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----------+
|        206|    William|    Gietz|  WGIETZ|515.123.8181|07-JUN-02|AC_ACCOUNT|  8300|            - |       205|          110|         1|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|          110|         1|
|        200|   Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|         1|
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|         1|
|        101|      Neena|  Kochhar|NKOCHH

In [0]:
df.withColumn("normal_rank",rank().over(win)).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-----------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|normal_rank|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-----------+
|        206|    William|    Gietz|  WGIETZ|515.123.8181|07-JUN-02|AC_ACCOUNT|  8300|            - |       205|          110|          1|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|          110|          1|
|        200|   Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|          1|
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|          1|
|        101|      Neena|  Kochhar

In [0]:
gr = Window.partitionBy(col("job_id"))
df.withColumn("sum",sum(col("salary")).over(gr))\
  .withColumn("avg",avg("salary").over(gr)).show()

+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-----+-------+
|EMPLOYEE_ID| FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|  sum|    avg|
+-----------+-----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-----+-------+
|        206|    William|    Gietz|  WGIETZ|515.123.8181|07-JUN-02|AC_ACCOUNT|  8300|            - |       205|          110| 8300| 8300.0|
|        205|    Shelley|  Higgins|SHIGGINS|515.123.8080|07-JUN-02|    AC_MGR| 12008|            - |       101|          110|12008|12008.0|
|        200|   Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10| 4400| 4400.0|
|        100|     Steven|     King|   SKING|515.123.4567|17-JUN-03|   AD_PRES| 24000|            - |        - |           90|24000|24000.0|
|        101|      N

In [0]:
#md Range and rows between

In [0]:
product_data = [
(2,"samsung","01-01-1995",11000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-01-2006",15000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(3,"oneplus","01-01-2010",23000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

product_df.show()


+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         2|     samsung|01-01-1995|  11000|
|         1|      iphone|01-02-2023|1300000|
|         2|     samsung|01-02-2023|1120000|
|         3|     oneplus|01-02-2023|1120000|
|         1|      iphone|01-03-2023|1600000|
|         2|     samsung|01-03-2023|1080000|
|         3|     oneplus|01-03-2023|1160000|
|         1|      iphone|01-01-2006|  15000|
|         1|      iphone|01-04-2023|1700000|
|         2|     samsung|01-04-2023|1800000|
|         3|     oneplus|01-04-2023|1170000|
|         1|      iphone|01-05-2023|1200000|
|         2|     samsung|01-05-2023| 980000|
|         3|     oneplus|01-05-2023|1175000|
|         1|      iphone|01-06-2023|1100000|
|         3|     oneplus|01-01-2010|  23000|
|         2|     samsung|01-06-2023|1100000|
|         3|     oneplus|01-06-2023|1200000|
+----------+------------+----------+-------+



In [0]:
#Find the difference in sales in for each product from their 1 st sales to the latest sales
win = Window.partitionBy(col("product_id")).orderBy(col("sales_date")).rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
product_df.withColumn("First Sales",first("sales").over(win)) \
           .withColumn("Last Sales",last("sales").over(win)) \
            .withColumn("Diff of sales",col("Last Sales") - col("First Sales")).show()                                                     

+----------+------------+----------+-------+-----------+----------+-------------+
|product_id|product_name|sales_date|  sales|First Sales|Last Sales|Diff of sales|
+----------+------------+----------+-------+-----------+----------+-------------+
|         1|      iphone|01-01-2006|  15000|      15000|   1100000|      1085000|
|         1|      iphone|01-02-2023|1300000|      15000|   1100000|      1085000|
|         1|      iphone|01-03-2023|1600000|      15000|   1100000|      1085000|
|         1|      iphone|01-04-2023|1700000|      15000|   1100000|      1085000|
|         1|      iphone|01-05-2023|1200000|      15000|   1100000|      1085000|
|         1|      iphone|01-06-2023|1100000|      15000|   1100000|      1085000|
|         2|     samsung|01-01-1995|  11000|      11000|   1100000|      1089000|
|         2|     samsung|01-02-2023|1120000|      11000|   1100000|      1089000|
|         2|     samsung|01-03-2023|1080000|      11000|   1100000|      1089000|
|         2|    

In [0]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

emp_df.show()

+---+------+----------+-----+
| id|  name|      date| time|
+---+------+----------+-----+
|  1|manish|11-07-2023|10:20|
|  1|manish|11-07-2023|11:20|
|  2|rajesh|11-07-2023|11:20|
|  1|manish|11-07-2023|11:50|
|  2|rajesh|11-07-2023|13:20|
|  1|manish|11-07-2023|19:20|
|  2|rajesh|11-07-2023|17:20|
|  1|manish|12-07-2023|10:32|
|  1|manish|12-07-2023|12:20|
|  3|vikash|12-07-2023|09:12|
|  1|manish|12-07-2023|16:23|
|  3|vikash|12-07-2023|18:08|
+---+------+----------+-----+



In [0]:
# Find emp who have not completed 8 hrs in office

emp_df = emp_df.withColumn("login",from_unixtime(unix_timestamp(concat(col("date"),lit(" "),col("time")),"dd-MM-yyyy HH:mm")))
win1 = Window.partitionBy(col("id"),col("date")).orderBy("login").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)      

In [0]:
emp_df.withColumn("flogin",to_timestamp(first("login").over(win1))) \
        .withColumn("llogin",to_timestamp(last("login").over(win1)))\
        .withColumn("diff in sec",col("llogin").cast("long") - col("flogin").cast("long")) \
        .withColumn("diff in hrs",round(col("diff in sec")/3600))\
        .filter(col("diff in hrs") <8 ) \
        .show()

+---+------+----------+-----+-------------------+-------------------+-------------------+-----------+-----------+
| id|  name|      date| time|              login|             flogin|             llogin|diff in sec|diff in hrs|
+---+------+----------+-----+-------------------+-------------------+-------------------+-----------+-----------+
|  1|manish|12-07-2023|10:32|2023-07-12 10:32:00|2023-07-12 10:32:00|2023-07-12 16:23:00|      21060|        6.0|
|  1|manish|12-07-2023|12:20|2023-07-12 12:20:00|2023-07-12 10:32:00|2023-07-12 16:23:00|      21060|        6.0|
|  1|manish|12-07-2023|16:23|2023-07-12 16:23:00|2023-07-12 10:32:00|2023-07-12 16:23:00|      21060|        6.0|
|  2|rajesh|11-07-2023|11:20|2023-07-11 11:20:00|2023-07-11 11:20:00|2023-07-11 17:20:00|      21600|        6.0|
|  2|rajesh|11-07-2023|13:20|2023-07-11 13:20:00|2023-07-11 11:20:00|2023-07-11 17:20:00|      21600|        6.0|
|  2|rajesh|11-07-2023|17:20|2023-07-11 17:20:00|2023-07-11 11:20:00|2023-07-11 17:20:00

In [0]:
# Find the performance of the sales in the last 3 months average. 
win = Window.partitionBy(col("product_id")).orderBy(col("sales_date")).rowsBetween(-2,0)
product_df.withColumn("Avg_sales_3", avg("sales").over(win)).show()

+----------+------------+----------+-------+------------------+
|product_id|product_name|sales_date|  sales|       Avg_sales_3|
+----------+------------+----------+-------+------------------+
|         1|      iphone|01-01-2006|  15000|           15000.0|
|         1|      iphone|01-02-2023|1300000|          657500.0|
|         1|      iphone|01-03-2023|1600000| 971666.6666666666|
|         1|      iphone|01-04-2023|1700000|1533333.3333333333|
|         1|      iphone|01-05-2023|1200000|         1500000.0|
|         1|      iphone|01-06-2023|1100000|1333333.3333333333|
|         2|     samsung|01-01-1995|  11000|           11000.0|
|         2|     samsung|01-02-2023|1120000|          565500.0|
|         2|     samsung|01-03-2023|1080000|          737000.0|
|         2|     samsung|01-04-2023|1800000|1333333.3333333333|
|         2|     samsung|01-05-2023| 980000|1286666.6666666667|
|         2|     samsung|01-06-2023|1100000|1293333.3333333333|
|         3|     oneplus|01-01-2010|  23